In [3]:
import gym
import keras
import datetime as dt
import tensorflow as tf
import random
import numpy as np
import math
from tensorflow.keras.layers import Dense

STORE_PATH = '/run'
MAX_EPSILON = 1
MIN_EPSILON = 0.01
LAMBDA = 0.0005
GAMMA = 0.95
BATCH_SIZE = 32
TAU = 0.08
RANDOM_REWARD_STD = 1.0
env = gym.make("CartPole-v1")
# env = gym.make("MountainCar-v0")
state_size = 4
num_actions = env.action_space.n



class Memory():
    def __init__(self, max_memory):
        self._max_memory = max_memory
        self._samples = []
    def add_sample(self, sample):
        self._samples.append(sample)
        if len(self._samples) > self._max_memory:
            self._samples.pop(0)
    def sample(self, no_samples):
        if no_samples > len(self._samples):
            return random.sample(self._samples, len(self._samples))
        else:
            return random.sample(self._samples, no_samples)
    @property
    def num_samples(self):
        return len(self._samples)
memory = Memory(50)


class QN(tf.keras.Model):
    def __init__(self):
        super(QN,self).__init__()
        self.l1 = Dense(30, input_shape=(4,), kernel_initializer='random_uniform',
                bias_initializer='random_uniform')
        self.l2 = Dense(30, kernel_initializer='random_uniform',
                bias_initializer='random_uniform')
        self.l3 = Dense(num_actions, kernel_initializer='random_uniform',
                bias_initializer='random_uniform')

    def call(self, input):
        feat = tf.nn.relu(self.l1(input))
        feat = tf.nn.relu(self.l2(feat))
        value = self.l3(feat)
        return value

primary_network = QN()
target_network = QN()


def choose_action(state, primary_network, eps):
    if random.random() < eps:
        return random.randint(0, num_actions - 1)
    else:
        state = np.expand_dims(np.array(state),axis=0) #otherwise throuhg eerror..
        return np.argmax(primary_network(state))
    
    
def train(primary_network, memory, tarket_network):
    if memory.num_samples < BATCH_SIZE*3:
        return 0
    batch = memory.sample(BATCH_SIZE)
    states = np.array(val[0] for val in batch)
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([(np.zeros(state_size)
                             if val[3] is None else val[3]) for val in batch])
   
    prim_qt = primary_network(np.expand_dims(states,axis=0)) # Q_t[s,a]
    prim_qtp1 = primary_network(np.expand_dims(next_states,axis=0)) #Q_{t+1}[s,a]
        
    updates = rewards
    valid_idxs = np.array(next_states).sum(axis=1) != 0
    batch_idxs = np.arange(BATCH_SIZE)
    
    opt_q_tp1_eachS = np.argmax(np.squeeze(prim_qtp1.numpy()), axis=0)
    q_from_target = target_network(np.expand_dims(next_states), axis=0)
    
    updates[valid_idx] += GAMMA*np.squeeze(q_from_target.numpy())[valid_idx, opt_q_tp1_eachS[valid_idx]]
    
    target_q = np.squeeze(prim_qt.numpy())
    target_q[batch_idxs, actions] = updates

    with tf.device("/cpu:0"):
        with tf.GradientTape() as tape:
            tape.watch(primary_network.trainable_variables)
            predicted_q = primary_network(states)
            target_q = np.expand_dims(target_q,axis=0)
            loss = tf.keras.losses.MSE(predicted_q, target_q)
            loss = tf.reduce_mean(loss)

            grads = tape.gradient(loss, primary_network.trainable_variables)
            optimizer.apply_gradients(zip(grads, primary_network.trainable_variables))

    for t, e in zip(target_network.trainable_variables, primary_network.trainable_variables):
        t.assign(t*(1-TAU) + e*TAU)

    return loss
    


In [2]:
def train(primary_network, memory, tarket_network):
    if memory.num_samples < BATCH_SIZE*3:
        return 0
    batch = memory.sample(BATCH_SIZE)
    states = np.array(val[0] for val in batch)
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([(np.zeros(state_size)
                             if val[3] is None else val[3]) for val in batch])
   
    prim_qt = primary_network(np.expand_dims(states,axis=0)) # Q_t[s,a]
    prim_qtp1 = primary_network(np.expand_dims(next_states,axis=0)) #Q_{t+1}[s,a]
        
    updates = rewards
    valid_idxs = np.array(next_states).sum(axis=1) != 0
    batch_idxs = np.arange(BATCH_SIZE)
    
    opt_q_tp1_eachS = np.argmax(np.squeeze(prim_qtp1.numpy()), axis=0)
    q_from_target = target_network(np.expand_dims(next_states), axis=0)
    
    updates[valid_idx] += GAMMA*np.squeeze(q_from_target.numpy())[valid_idx, opt_q_tp1_eachS[valid_idx]]
    
    target_q = np.squeeze(prim_qt.numpy())
    target_q[batch_idxs, actions] = updates

    with tf.device("/cpu:0"):
        with tf.GradientTape() as tape:
            tape.watch(primary_network.trainable_variables)
            predicted_q = primary_network(states)
            target_q = np.expand_dims(target_q,axis=0)
            loss = tf.keras.losses.MSE(predicted_q, target_q)
            loss = tf.reduce_mean(loss)

            grads = tape.gradient(loss, primary_network.trainable_variables)
            optimizer.apply_gradients(zip(grads, primary_network.trainable_variables))

    for t, e in zip(target_network.trainable_variables, primary_network.trainable_variables):
        t.assign(t*(1-TAU) + e*TAU)

    return loss
    

In [17]:
state = env.reset()
for i in range(60):
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    memory.add_sample((state, action, reward, next_state))
    state = next_state

/home/cooper-cooper/.local/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [18]:
batch = memory.sample(10)

In [29]:
states = np.array([val[0] for val in batch])
actions = np.array([val[1] for val in batch])
rewards = np.array([val[2] for val in batch])
next_states = np.array([(np.zeros(state_size)
                             if val[3] is None else val[3]) for val in batch])

In [31]:
states

array([[-0.97836618, -2.98492829,  1.30600539,  6.10198808],
       [ 0.07197702,  0.02238607, -0.1029633 , -0.22086573],
       [-1.56228939, -2.51704135,  2.81357344,  9.11456021],
       [ 0.08232229, -0.16419128, -0.14357026, -0.11668926],
       [ 0.07189856, -0.54975871, -0.14335451,  0.37080563],
       [ 0.06090338, -0.742584  , -0.13593839,  0.61507463],
       [-1.40545012, -2.48158903,  2.29619606,  8.55503281],
       [-0.23000408, -1.71324798,  0.16478118,  1.9842409 ],
       [ 0.07680109,  0.02535539, -0.11826402, -0.28715771],
       [-0.84647728, -3.40053012,  1.0703208 ,  5.83750069]])

In [32]:
prim_qt = primary_network(np.expand_dims(states,axis=0)) # Q_t[s,a]
prim_qtp1 = primary_network(np.expand_dims(next_states,axis=0)) #Q_{t+1}[s,a]


W1030 11:57:44.159261 139623576430400 base_layer.py:1814] Layer qn_2 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [33]:
prim_qt

<tf.Tensor: id=124, shape=(1, 10, 2), dtype=float32, numpy=
array([[[ 0.02813783, -0.00222196],
        [ 0.02857228, -0.00535921],
        [ 0.02770496, -0.00064244],
        [ 0.02851255, -0.00530304],
        [ 0.02832131, -0.00509667],
        [ 0.02823165, -0.00498947],
        [ 0.02783949, -0.00074054],
        [ 0.02813262, -0.00456427],
        [ 0.02859422, -0.00536845],
        [ 0.02805543, -0.00271018]]], dtype=float32)>

In [34]:
prim_qtp1

<tf.Tensor: id=170, shape=(1, 10, 2), dtype=float32, numpy=
array([[[ 0.02814264, -0.00206854],
        [ 0.02866961, -0.00539125],
        [ 0.02755707, -0.0006103 ],
        [ 0.0284155 , -0.00520371],
        [ 0.02823165, -0.00498947],
        [ 0.02820428, -0.00487988],
        [ 0.02784084, -0.000725  ],
        [ 0.02808827, -0.00447877],
        [ 0.02868405, -0.0053953 ],
        [ 0.02809866, -0.00247803]]], dtype=float32)>

In [36]:
updates = rewards
updates

array([0., 1., 0., 1., 1., 1., 0., 1., 1., 0.])

In [38]:
valid_idxs = np.array(next_states).sum(axis=1) != 0
valid_idxs

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [40]:
batch_idxs = np.arange(10)

In [44]:
opt_q_tp1_eachS = np.argmax(np.squeeze(prim_qtp1.numpy()), axis=1)


In [45]:
opt_q_tp1_eachS

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [47]:
q_from_target = target_network(np.expand_dims(next_states, axis=0))

W1030 12:06:32.241358 139623576430400 base_layer.py:1814] Layer qn_3 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [48]:
q_from_target

<tf.Tensor: id=294, shape=(1, 10, 2), dtype=float32, numpy=
array([[[ 0.01157035, -0.04272651],
        [ 0.01184477, -0.04194078],
        [ 0.01122182, -0.04295984],
        [ 0.01174406, -0.04222467],
        [ 0.01182225, -0.04239814],
        [ 0.01187006, -0.04246003],
        [ 0.01126141, -0.04278949],
        [ 0.01190042, -0.04264418],
        [ 0.01185436, -0.04194633],
        [ 0.01167748, -0.04278158]]], dtype=float32)>

In [52]:
np.squeeze(q_from_target.numpy())[valid_idxs,:]

array([[ 0.01157035, -0.04272651],
       [ 0.01184477, -0.04194078],
       [ 0.01122182, -0.04295984],
       [ 0.01174406, -0.04222467],
       [ 0.01182225, -0.04239814],
       [ 0.01187006, -0.04246003],
       [ 0.01126141, -0.04278949],
       [ 0.01190042, -0.04264418],
       [ 0.01185436, -0.04194633],
       [ 0.01167748, -0.04278158]], dtype=float32)

In [54]:
np.squeeze(q_from_target.numpy())[valid_idxs,opt_q_tp1_eachS[valid_idxs]]

array([0.01157035, 0.01184477, 0.01122182, 0.01174406, 0.01182225,
       0.01187006, 0.01126141, 0.01190042, 0.01185436, 0.01167748],
      dtype=float32)

In [58]:
updates[valid_idxs] += GAMMA*np.squeeze(q_from_target.numpy())[valid_idxs, opt_q_tp1_eachS[valid_idxs]]

In [64]:

target_q = np.squeeze(prim_qt.numpy())
target_q[batch_idxs, actions] = updates


In [65]:
target_q

array([[ 1.09918285e-02, -2.22195592e-03],
       [ 2.85722818e-02,  1.01125252e+00],
       [ 2.77049579e-02,  1.06607312e-02],
       [ 1.01115680e+00, -5.30303875e-03],
       [ 1.01123118e+00, -5.09666838e-03],
       [ 1.01127660e+00, -4.98947175e-03],
       [ 1.06983380e-02, -7.40542077e-04],
       [ 1.01130545e+00, -4.56427224e-03],
       [ 2.85942201e-02,  1.01126170e+00],
       [ 2.80554295e-02,  1.10936053e-02]], dtype=float32)

In [75]:
optimizer = tf.keras.optimizers.Adam(lr=0.01)
